In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from operator import itemgetter

loader = PDFPlumberLoader("./data/SPRI_AI_Brief_2023년12월호_F.pdf")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=30)
documents = text_splitter.split_documents(docs)

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = FAISS.from_documents(documents, embeddings)
retriever = vectorstore.as_retriever()

llm = ChatOpenAI(model="gpt-4o-mini")

prompt = hub.pull("teddynote/rag-korean-with-source")

chain = ({"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
        )

In [5]:
chain.invoke("삼성전자 생성형 AI의 이름은?")

'삼성전자 생성형 AI의 이름은 ‘삼성 가우스’입니다. 이 AI는 언어, 코드, 이미지의 3개 모델로 구성되어 있습니다. 출처: 삼성전자, ‘삼성 AI 포럼’서 자체 개발 생성형 AI ‘삼성 가우스’ 공개, 2023.11.08.'

In [23]:
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

from operator import itemgetter

loader = PDFPlumberLoader("./data/SPRI_AI_Brief_2023년12월호_F.pdf")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=30)
documents = text_splitter.split_documents(docs)

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = FAISS.from_documents(documents, embeddings)
retriever = vectorstore.as_retriever()

llm = ChatOpenAI(model="gpt-4o-mini")

prompt = hub.pull("teddynote/rag-korean-with-source")

chain = ({"context": itemgetter("question")|retriever, "question": itemgetter("question")}
        | prompt
        | llm
        | StrOutputParser()
        )

chain.invoke({"question": "삼성전자 생성형 ai의 이름은?"})

"삼성전자의 생성형 AI 이름은 '삼성 가우스'입니다. 이 AI는 언어, 코드, 이미지의 3개 모델로 구성되어 있으며, 온디바이스에서 작동하도록 설계되었습니다. 출처: 삼성전자, ‘삼성 AI 포럼’서 자체 개발 생성형 AI ‘삼성 가우스’ 공개, 2023.11.08."